# **Esquema de comunicacion**

### **Menú**
* Codificacion Huffman
* Codificacion Shannon
* Codificacion LZW
* Codificacion RLE

En esta V9 se agrega la simulacion de perdida y recuperacion de paquetes mas no se implementa la "Modulacion dinamica"

### **Función principal para leer el archivo .txt**

In [1]:
def leer_archivo(nombre_archivo):

    try:
        with open(nombre_archivo, 'r') as archivo:
            contenido = archivo.read()
            return contenido
    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no fue encontrado.")
        return ""

### **Codificación Huffman**

In [2]:
def texto_a_binario(mensaje):
    binario = ''.join(format(ord(char), '08b') for char in mensaje)
    return binario

class Nodo:
    def __init__(self, char, freq, izq=None, der=None):
        self.char = char
        self.freq = freq
        self.izq = izq
        self.der = der

def arbol(texto):
    freq = dict()
    for char in texto:
        if char not in freq:
            freq[char] = 0
        freq[char] += 1

    nodos = [Nodo(char, f) for char, f in freq.items()]
    while len(nodos) > 1:
        nodos = sorted(nodos, key=lambda x: x.freq)  # Ordena los nodos por frecuencia
        izq = nodos.pop(0)
        der = nodos.pop(0)
        nodo = Nodo(None, izq.freq + der.freq, izq, der)
        nodos.append(nodo)
    return nodos[0]  # Nodo raíz

def diccionario_huffman(nodo, codigo_binario="", diccionario=dict()):
    if nodo is None:
        return
    if nodo.char is not None:
        diccionario[nodo.char] = codigo_binario
    diccionario_huffman(nodo.izq, codigo_binario + "0", diccionario)
    diccionario_huffman(nodo.der, codigo_binario + "1", diccionario)
    return diccionario

def decodificar(texto_codificado, nodo_raiz):
    texto = ""
    nodo_actual = nodo_raiz
    for bit in texto_codificado:
        if bit == '0':
            nodo_actual = nodo_actual.izq
        else:  # bit == '1'
            nodo_actual = nodo_actual.der
        if nodo_actual.char is not None:  # Es un nodo hoja
            texto += nodo_actual.char
            nodo_actual = nodo_raiz
    return texto

### **Codificación Shannon**

In [3]:
# Paso 1: Convertir el mensaje de texto a una cadena binaria
def texto_a_binario(mensaje):
    binario = ''.join(format(ord(char), '08b') for char in mensaje)
    return binario

# Paso 2: Dividir la cadena binaria en bytes (8 bits)
def dividir_cadena(binario):
    return [binario[i:i+8] for i in range(0, len(binario), 8)]

# Paso 3: Calcular las frecuencias de cada byte
from collections import defaultdict

def calcular_frecuencias(bytes):
    frecuencias = defaultdict(int)
    for byte in bytes:
        frecuencias[byte] += 1
    return frecuencias

# Paso 4: Construir la tabla de codificación Shannon-Fano
def construir_tabla(frecuencias):
    # Ordenar las frecuencias en orden descendente
    frecuencias_ordenadas = sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)

    # Crear la tabla de codificación Shannon-Fano
    tabla_codificacion = {}
    def asignar_codigos(frecuencias_ordenadas, codigo=''):
        if len(frecuencias_ordenadas) == 1:
            tabla_codificacion[frecuencias_ordenadas[0][0]] = codigo
        else:
            split_point = len(frecuencias_ordenadas) // 2
            for byte, frecuencia in frecuencias_ordenadas[:split_point]:
                tabla_codificacion[byte] = codigo + '0'
            for byte, frecuencia in frecuencias_ordenadas[split_point:]:
                tabla_codificacion[byte] = codigo + '1'
            asignar_codigos(frecuencias_ordenadas[:split_point], codigo + '0')
            asignar_codigos(frecuencias_ordenadas[split_point:], codigo + '1')

    asignar_codigos(frecuencias_ordenadas)
    return tabla_codificacion

# Paso 5: Codificar el mensaje utilizando la tabla de codificación Shannon-Fano
def codificacion_shannon(bytes_binarios, tabla_codificacion):
    texto_codificado = ''.join(tabla_codificacion[byte] for byte in bytes_binarios)
    return texto_codificado


# Paso 6: Decodificar el texto codificado utilizando la tabla de codificación Shannon-Fano
def decodificacion_shannon(texto_codificado, tabla_codificacion):
    texto_decodificado = ''
    codigo_actual = ''
    for bit in texto_codificado:
        codigo_actual += bit
        if codigo_actual in tabla_codificacion.values():
            byte = [key for key, value in tabla_codificacion.items() if value == codigo_actual][0]
            texto_decodificado += byte
            codigo_actual = ''
    return texto_decodificado

# Función para convertir una cadena binaria en texto
def binario_a_texto(binario):
    texto = ""
    for i in range(0, len(binario), 8):
        byte = binario[i:i+8]
        char = chr(int(byte, 2))
        texto += char
    return texto

### **Códificación Lempel Ziv y Welch**

In [4]:
def comprimir(texto):
    diccionario = {chr(i): i for i in range(256)}  # Inicializar el diccionario con caracteres ASCII
    siguiente_codigo = 256
    resultado = []
    codigo_actual = ""

    for caracter in texto:
        codigo_actual += caracter
        if codigo_actual not in diccionario:
            diccionario[codigo_actual] = siguiente_codigo
            siguiente_codigo += 1
            resultado.append(diccionario[codigo_actual[:-1]])
            codigo_actual = caracter

    resultado.append(diccionario[codigo_actual])  # Agregar el último código

    return resultado

def descomprimir(datos_comprimidos):
    diccionario = {i: chr(i) for i in range(256)}  # Inicializar el diccionario con caracteres ASCII
    siguiente_codigo = 256
    resultado = []
    codigo_actual = chr(datos_comprimidos[0])
    resultado.append(codigo_actual)

    for codigo in datos_comprimidos[1:]:
        if codigo in diccionario:
            entrada = diccionario[codigo]
        elif codigo == siguiente_codigo:
            entrada = codigo_actual + codigo_actual[0]
        else:
            raise ValueError("Datos inválidos.")

        resultado.append(entrada)
        diccionario[siguiente_codigo] = codigo_actual + entrada[0]
        siguiente_codigo += 1
        codigo_actual = entrada

    print("Diccionario:")
    for clave, valor in diccionario.items():
        print(f"{clave}: {valor}")

    return ''.join(resultado)

### **Codificación Run-Length Encoding**

In [5]:
def codificacion_RLE(texto):
    texto_codificado = []
    conteo = 1

    for i in range(1, len(texto)):
        if texto[i] == texto[i - 1]:
            conteo += 1
        else:
            texto_codificado.append(texto[i - 1] + str(conteo))
            conteo = 1

    texto_codificado.append(texto[-1] + str(conteo))

    return "".join(texto_codificado)

def decodificacion_RLE(texto_codificado):
    texto_decodificado = []
    i = 0

    while i < len(texto_codificado):
        char = texto_codificado[i]
        i += 1
        count_str = ""
        while i < len(texto_codificado) and texto_codificado[i].isdigit():
            count_str += texto_codificado[i]
            i += 1
        count = int(count_str)
        texto_decodificado.append(char * count)

    return "".join(texto_decodificado)

def diccionario_RLE(texto):
    conjunto = set(texto)
    print("Diccionario implícito de RLE:")
    for caracter in conjunto:
        print(f"'{caracter}' -> {caracter}")

### Ruido (Es la misma de la V8...)

Ruido simple cambiando 1 por 0 aleatoriamente.

In [6]:
import random

probabilidad_de_error = random.random()

def canal_con_ruido(texto_codificado, probabilidad_ruido):
    texto_con_ruido = ""

    for bit in texto_codificado:
        if random.random() < probabilidad_ruido:
            # Invertir el bit (de 0 a 1 o de 1 a 0)
            bit = '1' if bit == '0' else '0'

        texto_con_ruido += bit

    return texto_con_ruido

def agregar_ruido(signal, probabilidad):

    signal_con_ruido = ""

    for bit in signal:
        if random.random() < probabilidad:
            # Invierte el bit con la probabilidad dada
            bit = '0' if bit == '1' else '1'

        signal_con_ruido += bit

    return signal_con_ruido

def ruido_LZW(codigos_lzw, probabilidad):

    codigos_con_ruido = []
    # Crear una lista de códigos entre 40 y 125
    codigos_lzw_rango = list(range(41, 125))


    for codigo in codigos_lzw:
        if random.random() < probabilidad:
            # Invierte el bit con la probabilidad dada
            # codigo_corrupto = 120
            # Seleccionar un código aleatorio de la lista
            codigo_corrupto = random.choice(codigos_lzw_rango)

            codigos_con_ruido.append(codigo_corrupto)
        else:
            codigos_con_ruido.append(codigo)

    return codigos_con_ruido

def ruido_RLE(cadena_codificada, probabilidad):
    rango_error = random.random()

    while rango_error < probabilidad:
        # print("W Rango de error:",rango_error)
        # print("W probabilidad de error:",probabilidad)
        rango_error = random.random()

    if rango_error < probabilidad:
        # print("X Rango de error:",rango_error)
        # print("X probabilidad de error:",probabilidad)
        raise ValueError("La probabilidad debe estar en el rango de 0 a 1")

    # print("S Rango de error:",rango_error)
    # print("S probabilidad de error:",probabilidad)

    cadena_con_ruido = list(cadena_codificada)

    for i in range(len(cadena_con_ruido)):
        if random.random() < probabilidad:
            # Invierte el carácter (cambia '0' a '1' o viceversa)
            if cadena_con_ruido[i] == '0':
                cadena_con_ruido[i] = '1'
            else:
                cadena_con_ruido[i] = '0'

    return ''.join(cadena_con_ruido)

def ruido_RLE2(texto, probabilidad):
    texto_con_ruido = list(texto)
    for i in range(len(texto_con_ruido)):
        if random.random() < probabilidad:
            # Cambia el carácter en la posición i a un carácter aleatorio
            texto_con_ruido[i] = chr(random.randint(32, 126))  # Caracteres imprimibles ASCII
    return ''.join(texto_con_ruido)

### Ruido (Perdida de paquetes para la V9)

In [7]:
import random

#############################################################################
#HUFFMAN

def dividir_paquetes(binario, longitud_paquete):
    paquetes = [binario[i:i+longitud_paquete] for i in range(0, len(binario), longitud_paquete)]
    return paquetes

def package_lost(paquetes, probabilidad_perdida):
    paquetes_perdidos = []
    paquetes_recibidos = []
    for paquete in paquetes:
        if random.random() < probabilidad_perdida:
            paquetes_perdidos.append(paquete)
        else:
            paquetes_recibidos.append(paquete)
    return paquetes_perdidos, paquetes_recibidos

def reconstruir_mensaje(paquetes_recibidos):
    mensaje_recibido = ''.join(paquetes_recibidos)
    return mensaje_recibido

#############################################################################
# SHANNON FANO

# Paso 1: Dividir el mensaje codificado en paquetes
def dividir_en_paquetes(mensaje_codificado, tamano_paquete):
    paquetes = [mensaje_codificado[i:i + tamano_paquete] for i in range(0, len(mensaje_codificado), tamano_paquete)]
    return paquetes

# Función para simular la pérdida de paquetes
def simular_perdida_de_paquetes(paquetes, probabilidad_perdida):
    paquetes_perdidos = []
    for paquete in paquetes:
        if random.random() < probabilidad_perdida:
            paquetes_perdidos.append(None)
            print(f"Paquete {paquete} se perdió. Retransmitiendo...")
        else:
            paquetes_perdidos.append(paquete)
    return paquetes_perdidos

# Función para recibir los paquetes y solicitar reenvío de los perdidos
def canal_con_perdidas(paquetes_perdidos, mensaje_codificado):
    nuevo_mensaje = []

    for paquete in paquetes_perdidos:
        if paquete is None:
            # Si el paquete se perdió, solicitar reenvío
            indice = len(nuevo_mensaje)
            while paquete is None:
                paquete = mensaje_codificado[indice]  # Simula el reenvío
                nuevo_mensaje.append(paquete)
                indice += 1
        else:
            nuevo_mensaje.append(paquete)

    return nuevo_mensaje

#############################################################################
# LZW

# Canal con pérdida de paquetes simulada
def canal_con_perdidas2(data, loss_probability):
    for i in range(len(data)):
        if random.random() < loss_probability:
            data[i] = None  # Representa la pérdida del paquete.
    return data

# Transceptor (Transmisor/Receptor)
def transceptor(data, texto, codificacion):
    for i in range(len(data)):
        while data[i] is None:  # Si un paquete se perdió...
            print(f"Paquete {i + 1} se perdió. Retransmitiendo...")
            data[i] = codificacion(texto)[i]  # Se vueve a transmitir el paquete perdido.
    return data

#############################################################################
# RLE

def codificacion_RLE_LOSS(text):
    encoded_text = []
    count = 1

    for i in range(1, len(text)):
        if text[i] == text[i - 1]:
            count += 1
        else:
            encoded_text.append(text[i - 1] + str(count))
            count = 1

    encoded_text.append(text[-1] + str(count))

    return encoded_text

def decodificacion_RLE_LOSS(encoded_text):
    decoded_text = []
    while len(encoded_text) > 0:
        current_item = encoded_text.pop(0)
        decoded_text.append(int(current_item[1:])*current_item[0])
    return ''.join(decoded_text)

## Menú

In [8]:
# Obtener la fuente de información (puedes leer un archivo o ingresar texto manualmente)
nombre_archivo_fuente_1 = "fuente 1.txt"
nombre_archivo_fuente_2 = "fuente 2.txt"
nombre_archivo_fuente_3 = "fuente 3.txt"
texto_original = leer_archivo(nombre_archivo_fuente_1)
texto_original_2 = leer_archivo(nombre_archivo_fuente_2)
texto_original_3= leer_archivo(nombre_archivo_fuente_3)
probabilidad_de_error = random.random()

# Menú para seleccionar el algoritmo de compresión
while True:
    print("|------------------------------------------|")
    print("| Seleccione un algoritmo de compresión:   |")
    print("|------------------------------------------|")
    print("| 1. Huffman                               |")
    print("| 2. Shannon-Fano                          |")
    print("| 3. LZW                                   |")
    print("| 4. RLE                                   |")
    print("| 0. Salir                                 |")
    print("|------------------------------------------|")
    opcion = int(input("Opción: "))

    if opcion == 0:
        print()
        print("╔═══════════════════════════╗")
        print("║         Saliendo...       ║")
        print("╚═══════════════════════════╝")
        print()
        break
    elif opcion == 1:
        print()
        print("|------------------------|")
        print("|  Codificación Huffman  |")
        print("|------------------------|")
        print()
        texto_binario = texto_a_binario(texto_original)
        raiz = arbol(texto_original)
        diccionario = diccionario_huffman(raiz)
        texto_codificado = "".join(diccionario[char] for char in texto_original)
        texto_decodificado = decodificar(texto_codificado, raiz)
        print("Cadena binaria original:",texto_binario)
        print("Cadena binaria comprimida:",texto_codificado)
        print("")
        print("Mensaje enviado:",texto_original)
        print("Mensaje recibido:",texto_decodificado)
        print("")
        print("Lista de simbolos")
        print(diccionario)
        print()
        ruido = int(input("¿Quieres agregar ruido? 1/0"))
        if ruido == 1:
            probabilidad_de_error = random.random()
            print("--------------> Mensaje con ruido <---------------\n")
            print("╔═════════════════════════════════════════════════╗")
            print("║ Probabilidad de error:",probabilidad_de_error*100,"%")
            print("╚═════════════════════════════════════════════════╝")
            print()
            print("Mensaje:", texto_original)
            texto_binario = texto_a_binario(texto_original)
            longitud_paquete = 8  # Define la longitud del paquete
            paquetes = dividir_paquetes(texto_binario, longitud_paquete)
            probabilidad_perdida = 0.4  # Define la probabilidad de pérdida de paquetes
            paquetes_perdidos, paquetes_recibidos = package_lost(paquetes, probabilidad_de_error)
            mensaje_recibido = reconstruir_mensaje(paquetes_recibidos)
            texto_decodificado = decodificar(mensaje_recibido, raiz)
            mensaje_recibido = reconstruir_mensaje(paquetes_recibidos)
            texto_decodificado = decodificar(mensaje_recibido, raiz)
            # 3. Canal (Simulación de pérdida de paquetes)
            print()
            print("Paquetes enviados:", len(paquetes))
            print("Paquetes perdidos:", len(paquetes_perdidos))
            print("Paquetes recibidos:", len(paquetes_recibidos))
            print()
            # 5. Destino de información
            print("Cadena binaria original:", texto_binario)
            print("Cadena binaria recibida:", mensaje_recibido)
            print()
            print("Mensaje enviado:", texto_original)
            print("Mensaje recibido con perdidas:", texto_decodificado)
            # Lista de símbolos
            print()
            print("Lista de símbolos")
            print()
            for byte, codigo in diccionario.items():
                print(f"{byte} = {codigo}")
            print()
            # texto_con_ruido = canal_con_ruido(texto_codificado, probabilidad_de_error)
            # texto_decodificado = decodificar(texto_con_ruido, raiz)
            # print("Cadena binaria original:",texto_binario)
            # print("Cadena binaria comprimida con ruido:",texto_con_ruido)
            # print("")
            # print("Mensaje enviado:",texto_original)
            # print("Mensaje recibido:",texto_decodificado)
        elif ruido == 0:
            print("___Sin ruido___\n")
    elif opcion == 2:
        print()
        print("|------------------------|")
        print("|  Codificación Shannon  |")
        print("|------------------------|")
        print()
        mensaje_binario = texto_a_binario(texto_original)
        bytes_binarios = dividir_cadena(mensaje_binario)
        frecuencias = calcular_frecuencias(bytes_binarios)
        tabla_codificacion_sf = construir_tabla(frecuencias)
        mensaje_codificado = codificacion_shannon(bytes_binarios, tabla_codificacion_sf)
        mensaje_decodificado = decodificacion_shannon(mensaje_codificado, tabla_codificacion_sf)
        print("Mensaje Codificado  :", mensaje_codificado)
        print("Mensaje Decodificado:", mensaje_decodificado)
        # Decodificar el mensaje y convertirlo a texto
        mensaje_binario_decodificado = mensaje_decodificado
        mensaje_texto_decodificado = binario_a_texto(mensaje_binario_decodificado)
        # Comparar el mensaje decodificado con el texto original
        print()
        print("Mensaje original:", texto_original)
        print("Mensaje recivido:",mensaje_texto_decodificado)
        if mensaje_texto_decodificado == texto_original:
            print()
            print("<-- El mensaje decodificado coincide con el texto original -->")
        else:
            print()
            print("--> El mensaje decodificado no coincide con el texto original <--")
        # Asegurar que todos los códigos tengan 5 dígitos
        for byte, codigo in tabla_codificacion_sf.items():
            while len(codigo) < 5:
                codigo += '0'  # Agrega ceros al final hasta que tenga 5 dígitos
            tabla_codificacion_sf[byte] = codigo
        # Imprimir la tabla de codificación actualizada
        print()
        print("|-----------------------|")
        print("| Tabla de codificación |")
        print("|-----------------------|")
        for byte, codigo in tabla_codificacion_sf.items():
            print(f"| {byte}  | {codigo}     |")
        print("|-----------------------|")
        print()
        ruido = int(input("¿Quieres agregar ruido? 1/0"))
        if ruido == 1:
            probabilidad_de_error = random.uniform(0.3, 0.8)
            print("---------------> Mensaje con ruido <---------------\n")
            print("╔═════════════════════════════════════════════════╗")
            print("║ Probabilidad de error:",probabilidad_de_error*100,"%")
            print("╚═════════════════════════════════════════════════╝")
            print()
            print("Mensaje:", texto_original)
            print()
            mensaje_binario = texto_a_binario(texto_original)
            print("Mensaje binario:",mensaje_binario)
            bytes_binarios = dividir_cadena(mensaje_binario)
            print()
            print("Bytes binarios:",bytes_binarios)
            frecuencias = calcular_frecuencias(bytes_binarios)
            print()
            tabla_codificacion_sf = construir_tabla(frecuencias)
            mensaje_codificado = codificacion_shannon(bytes_binarios, tabla_codificacion_sf)
            print("Mensaje codificado:", mensaje_codificado)
            print()
            tamano_paquete = 8  # Define el tamaño de cada paquete
            paquetes = dividir_en_paquetes(mensaje_codificado, tamano_paquete)
            print("Paquetes: ", paquetes)
            print()
            paquetes_perdidos = simular_perdida_de_paquetes(paquetes, probabilidad_de_error)
            print()
            print("Paquetes perdidos:",paquetes_perdidos)
            print()
            nuevo_mensaje = canal_con_perdidas(paquetes_perdidos, paquetes)
            print("Nuevo mensaje recuperado:",nuevo_mensaje)
            mensaje_decodificado = decodificacion_shannon(mensaje_codificado, tabla_codificacion_sf)
            # print("Mensaje decodificado:", mensaje_decodificado)
            # Paso 6: Decodificar el mensaje recibido
            nuevo_mensaje_decodificado = decodificacion_shannon(''.join(nuevo_mensaje), tabla_codificacion_sf)
            print("Nuevo mensaje decodificado:",nuevo_mensaje_decodificado)
            mensaje_binario_decodificado = nuevo_mensaje_decodificado
            mensaje_texto_decodificado = binario_a_texto(mensaje_binario_decodificado)
            print()
            print("Mensaje recivido:",mensaje_texto_decodificado)
            # Comparar el mensaje decodificado con el texto original
            if mensaje_texto_decodificado == texto_original:
                print()
                print(" <-- El mensaje decodificado coincide con el texto original -->")
            else:
                print()
                print("--> El mensaje decodificado no coincide con el texto original <--")
            # Asegurar que todos los códigos tengan 5 dígitos
            for byte, codigo in tabla_codificacion_sf.items():
                while len(codigo) < 5:
                    codigo += '0'  # Agrega ceros al final hasta que tenga 5 dígitos
                tabla_codificacion_sf[byte] = codigo
            # Imprimir la tabla de codificación actualizada
            print()
            print("|-----------------------|")
            print("| Tabla de codificación |")
            print("|-----------------------|")
            for byte, codigo in tabla_codificacion_sf.items():
                print(f"| {byte}  | {codigo}     |")
            print("|-----------------------|")
            print()
        elif ruido == 0:
            print("___Sin ruido___\n")
    elif opcion == 3:
        print()
        print("|------------------------|")
        print("|    Codificación LZW    |")
        print("|------------------------|")
        print()
        compressed_data = comprimir(texto_original_2)
        print("Texto original:", texto_original_2)
        print("Datos comprimidos:", compressed_data)
        print()
        decompressed_text = descomprimir(compressed_data)
        print()
        print("Texto original:", texto_original_2)
        print("Texto recivido:", decompressed_text)
        print()
        ruido = int(input("¿Quieres agregar ruido? 1/0"))
        if ruido == 1:
            probabilidad_de_error = random.uniform(0.2, 0.8)
            print("---------------> Mensaje con ruido <---------------\n")
            print("╔═════════════════════════════════════════════════╗")
            print("║ Probabilidad de error:",probabilidad_de_error*100,"%")
            print("╚═════════════════════════════════════════════════╝")
            print()
            compressed_data = comprimir(texto_original_2)
            print("Datos comprimidos:", compressed_data)
            data_with_packet_loss = canal_con_perdidas2(compressed_data, probabilidad_de_error)
            print("Datos con pérdida de paquetes:", data_with_packet_loss)
            print()
            reliable_data = transceptor(data_with_packet_loss, texto_original_2, comprimir)
            print()
            print("Datos con retransmisión de paquetes perdidos:", reliable_data)
            decompressed_text = descomprimir(reliable_data)
            print()
            print("Texto original:", texto_original_2)
            print("Texto recivido:", decompressed_text)
            print()
        elif ruido == 0:
            print("___Sin ruido___\n")
    elif opcion == 4:
        print()
        print("|------------------------|")
        print("|    Codificación RLE    |")
        print("|------------------------|")
        print()
        encoded_text = codificacion_RLE(texto_original_3)
        decoded_text = decodificacion_RLE(encoded_text)
        print("Texto original:", texto_original_3)
        print()
        print("Texto codificado:", encoded_text)
        print("Texto decodificado:", decoded_text)
        print()
        diccionario_RLE(texto_original_3)
        print()
        ruido = int(input("¿Quieres agregar ruido? 1/0"))
        if ruido == 1:
            probabilidad_de_error = random.uniform(0.3, 0.9)
            print("---------------> Mensaje con ruido <---------------\n")
            print("╔═════════════════════════════════════════════════╗")
            print("║ Probabilidad de error:",probabilidad_de_error*100,"%")
            print("╚═════════════════════════════════════════════════╝")
            print()
            print("Texto original:", texto_original_3)
            print()
            # Comprimir el texto
            compressed_data_RLE = codificacion_RLE_LOSS(texto_original_3)
            print("Datos comprimidos:", compressed_data_RLE)
            # Simular perdida de paquetes en el canal
            data_with_packet_loss_RLE = canal_con_perdidas2(compressed_data_RLE, probabilidad_de_error)
            print("Datos con perdida de paquetes:", data_with_packet_loss_RLE)
            # Retransmisión de paquetes perdidos
            print()
            reliable_data_RLE = transceptor(data_with_packet_loss_RLE, texto_original_3, codificacion_RLE_LOSS)
            print()
            print("Datos con retransmisión de paquetes perdidos: ", reliable_data_RLE)
            print()
            print("Texto original:", texto_original_3)
            # Descompresión de los datos comprimidos
            decompressed_text_RLE = decodificacion_RLE_LOSS(reliable_data_RLE)
            print("Texto descomprimido:", decompressed_text_RLE)
            print()
        elif ruido == 0:
            print("___Sin ruido___\n")
    else:
        print()
        print("*** Opción no válida. Inténtalo de nuevo. ***")
        print()

|------------------------------------------|
| Seleccione un algoritmo de compresión:   |
|------------------------------------------|
| 1. Huffman                               |
| 2. Shannon-Fano                          |
| 3. LZW                                   |
| 4. RLE                                   |
| 0. Salir                                 |
|------------------------------------------|

|------------------------|
|  Codificación Huffman  |
|------------------------|

Cadena binaria original: 0100100001101111011011000110000100101100001000000110010101110011011101000110111100100000011001010111001100100000011101010110111000100000011011010110010101101110011100110110000101101010011001010010000001100100011001010010000001110000011100100111010101100101011000100110000100100000011100000110000101110010011000010010000001101100011000010010000001100011011011110110110101110101011011100110100101100011011000010110001101101001011011110110111000101110
Cadena binaria comprimida: 11010010011